In [1]:
import keras as k
from keras.layers import merge
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.callbacks import History
from keras.layers import Activation
from keras.models import model_from_json
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from scipy.ndimage import rotate as rot
from sklearn.model_selection import train_test_split
from keras import utils
import numpy as np
import tensorflow as tf


def my_autoencode(img_shape, code_size=32):
    H,W,C = img_shape
    
    # encoder
    encoder = k.models.Sequential()
    encoder.add(k.layers.InputLayer(img_shape))
    encoder.add(k.layers.Conv2D(32, (3, 3), activation='elu', padding='same'))
    encoder.add(k.layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(k.layers.Conv2D(64, (3, 3), activation='elu', padding='same'))
    encoder.add(k.layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(k.layers.Conv2D(64, (3, 3), activation='elu', padding='same'))
    encoder.add(k.layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(k.layers.Conv2D(128, (3, 3), activation='elu', padding='same'))
    encoder.add(k.layers.AveragePooling2D((2, 2), padding='same'))
    encoder.add(k.layers.Flatten())
    encoder.add(k.layers.Dense(512, activation='elu'))
    encoder.add(k.layers.Dense(256, activation='elu'))
    encoder.add(k.layers.Dense(code_size, activation='elu'))
    encoder.summary()

    # decoder
    decoder = k.models.Sequential()
    decoder.add(k.layers.InputLayer((code_size,)))
    decoder.add(k.layers.Dense(256, activation='elu'))
    decoder.add(k.layers.Dense(512, activation='elu'))
    decoder.add(k.layers.Dense(8192, activation='elu'))
    decoder.add(k.layers.Reshape((8, 8, 128)))
    decoder.add(k.layers.UpSampling2D((2, 2)))
    decoder.add(k.layers.Conv2DTranspose(128, kernel_size=(3, 3), activation='elu', padding='same'))
    decoder.add(k.layers.UpSampling2D((2, 2)))
    decoder.add(k.layers.Conv2DTranspose(64, kernel_size=(3, 3), activation='elu', padding='same'))
    decoder.add(k.layers.UpSampling2D((2, 2)))
    decoder.add(k.layers.Conv2DTranspose(64, kernel_size=(3, 3), activation='elu', padding='same'))
    decoder.add(k.layers.UpSampling2D((2, 2)))
    decoder.add(k.layers.Conv2DTranspose(32, kernel_size=(3, 3), activation='elu', padding='same'))
    decoder.add(k.layers.Conv2DTranspose(3, kernel_size=(3, 3), activation='elu', padding='same')) # Unsure about this
    decoder.summary()
    
    return encoder, decoder

all_data = np.load('images_dataset.npy')
X_train = all_data[:2723, :]
X_test = all_data[2723:, ]
shape = X_train[0].shape # Get from dataset
encoder, decoder = my_autoencode(shape, code_size=128)
inp = k.layers.Input(shape)
code = encoder(inp)
reconstruction = decoder(code)
autoencoder = k.models.Model(inputs=inp, outputs=reconstruction)
autoencoder.compile(optimizer="adamax", loss='mse')
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('hackathon_autoencoder.{epoch:02d}-{val_loss:.2f}.h5', verbose=1, save_best_only=True)
]


# If you want to resume from a checkpoint
#     import keras.backend as K
#     def reset_tf_session():
#         K.clear_session()
#         tf.reset_default_graph()
#         s = K.get_session()
#         return s
#     #### uncomment below to continue training from model checkpoint
#     #### every time epoch counter starts at 0, so you need to track epochs manually
#     from keras.models import load_model
#     s = reset_tf_session()
#     autoencoder = load_model("checkpoints/hackathon_autoencoder.78-508.84.h5")  # continue after epoch 0+1
#     encoder = autoencoder.layers[1]
#     decoder = autoencoder.layers[2]

# # Train Model
autoencoder.fit(x=X_train, y=X_train,
                validation_data=[X_test, X_test],
                epochs=200,
                batch_size=32,
                shuffle=True,
                callbacks = callbacks
               )

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
__________

Epoch 15/200
2723/2723 [==============================] - 315s 116ms/step - loss: 1583.3912 - val_loss: 1495.4940
Epoch 16/200
2723/2723 [==============================] - 306s 112ms/step - loss: 1520.6933 - val_loss: 1419.3602
Epoch 17/200
2723/2723 [==============================] - 304s 112ms/step - loss: 1455.6052 - val_loss: 1391.1137
Epoch 18/200
2723/2723 [==============================] - 303s 111ms/step - loss: 1407.9795 - val_loss: 1426.0926
Epoch 19/200
2723/2723 [==============================] - 308s 113ms/step - loss: 1381.4904 - val_loss: 1386.3723
Epoch 20/200
2723/2723 [==============================] - 304s 112ms/step - loss: 1353.0519 - val_loss: 1348.7784
Epoch 21/200
2723/2723 [==============================] - 304s 112ms/step - loss: 1310.5019 - val_loss: 1299.6212
Epoch 22/200
2723/2723 [==============================] - 314s 115ms/step - loss: 1290.0222 - val_loss: 1384.3536
Epoch 23/200
2723/2723 [==============================] - 314s 115ms/step - loss: 1275.5

Epoch 47/200
2723/2723 [==============================] - 296s 109ms/step - loss: 841.9878 - val_loss: 1020.0332
Epoch 48/200
2723/2723 [==============================] - 297s 109ms/step - loss: 818.2738 - val_loss: 1061.7959
Epoch 49/200
2723/2723 [==============================] - 297s 109ms/step - loss: 827.5643 - val_loss: 1042.8332
Epoch 50/200
2723/2723 [==============================] - 297s 109ms/step - loss: 805.0769 - val_loss: 1034.2648
Epoch 51/200
2723/2723 [==============================] - 297s 109ms/step - loss: 790.7428 - val_loss: 1019.2368
Epoch 52/200
2723/2723 [==============================] - 297s 109ms/step - loss: 779.1424 - val_loss: 984.1836
Epoch 53/200
2723/2723 [==============================] - 297s 109ms/step - loss: 775.3656 - val_loss: 1056.1677
Epoch 54/200
2723/2723 [==============================] - 297s 109ms/step - loss: 790.0740 - val_loss: 992.9357
Epoch 55/200
2723/2723 [==============================] - 297s 109ms/step - loss: 761.4370 - val_l

Epoch 82/200
2723/2723 [==============================] - 297s 109ms/step - loss: 577.5207 - val_loss: 954.3979
Epoch 83/200
2723/2723 [==============================] - 297s 109ms/step - loss: 581.6796 - val_loss: 966.2000
Epoch 84/200
2723/2723 [==============================] - 297s 109ms/step - loss: 572.9007 - val_loss: 962.9184
Epoch 85/200
2723/2723 [==============================] - 297s 109ms/step - loss: 560.3559 - val_loss: 930.7353
Epoch 86/200
2723/2723 [==============================] - 297s 109ms/step - loss: 543.0347 - val_loss: 941.7235
Epoch 87/200
2723/2723 [==============================] - 296s 109ms/step - loss: 550.0928 - val_loss: 935.0594
Epoch 88/200
2723/2723 [==============================] - 297s 109ms/step - loss: 554.3494 - val_loss: 973.8956
Epoch 89/200
2723/2723 [==============================] - 297s 109ms/step - loss: 552.6480 - val_loss: 967.4944
Epoch 90/200
2723/2723 [==============================] - 297s 109ms/step - loss: 552.1125 - val_loss: 9

Epoch 119/200
2723/2723 [==============================] - 297s 109ms/step - loss: 426.5314 - val_loss: 966.6099
Epoch 120/200
2723/2723 [==============================] - 296s 109ms/step - loss: 423.2940 - val_loss: 985.2424
Epoch 121/200
2723/2723 [==============================] - 297s 109ms/step - loss: 427.8734 - val_loss: 976.3655
Epoch 122/200
2723/2723 [==============================] - 297s 109ms/step - loss: 429.3739 - val_loss: 974.8788
Epoch 123/200
2723/2723 [==============================] - 297s 109ms/step - loss: 424.6106 - val_loss: 969.0575
Epoch 124/200
2723/2723 [==============================] - 297s 109ms/step - loss: 411.0250 - val_loss: 971.8157
Epoch 125/200
2723/2723 [==============================] - 296s 109ms/step - loss: 413.3152 - val_loss: 966.4376
Epoch 126/200
2723/2723 [==============================] - 297s 109ms/step - loss: 413.6142 - val_loss: 981.4499
Epoch 127/200
2723/2723 [==============================] - 297s 109ms/step - loss: 409.6383 - va

2723/2723 [==============================] - 297s 109ms/step - loss: 349.8317 - val_loss: 1026.7277
Epoch 157/200
2723/2723 [==============================] - 299s 110ms/step - loss: 357.5758 - val_loss: 1025.3214
Epoch 158/200
2723/2723 [==============================] - 298s 109ms/step - loss: 360.6673 - val_loss: 1007.9116
Epoch 159/200
2723/2723 [==============================] - 297s 109ms/step - loss: 345.1236 - val_loss: 1026.9686
Epoch 160/200
2723/2723 [==============================] - 298s 109ms/step - loss: 341.0043 - val_loss: 1020.3631
Epoch 161/200
2723/2723 [==============================] - 298s 109ms/step - loss: 343.7560 - val_loss: 1027.2995
Epoch 162/200
2723/2723 [==============================] - 297s 109ms/step - loss: 341.1933 - val_loss: 1027.8028
Epoch 163/200
2723/2723 [==============================] - 297s 109ms/step - loss: 337.8958 - val_loss: 1025.3633
Epoch 164/200
2723/2723 [==============================] - 297s 109ms/step - loss: 343.4843 - val_loss

2723/2723 [==============================] - 297s 109ms/step - loss: 318.2740 - val_loss: 1045.5874
Epoch 194/200
2723/2723 [==============================] - 297s 109ms/step - loss: 306.9699 - val_loss: 1058.3514
Epoch 195/200
2723/2723 [==============================] - 297s 109ms/step - loss: 298.4424 - val_loss: 1047.0505
Epoch 196/200
2723/2723 [==============================] - 297s 109ms/step - loss: 297.2806 - val_loss: 1058.1522
Epoch 197/200
2723/2723 [==============================] - 298s 109ms/step - loss: 300.0449 - val_loss: 1088.2746
Epoch 198/200
2723/2723 [==============================] - 298s 109ms/step - loss: 305.4849 - val_loss: 1048.2089
Epoch 199/200
2723/2723 [==============================] - 297s 109ms/step - loss: 306.5782 - val_loss: 1078.9992
Epoch 200/200
2723/2723 [==============================] - 297s 109ms/step - loss: 298.8727 - val_loss: 1047.6184
